#### Pandas批量拆分Excel与合并Excel
实例演示：
1. 将一个大Excel等份拆分成多个Excel
2. 将多个小Excel合并成一个大Excel，并标记来源

In [1]:
work_dir='./files/excel_files'
splits_dir=f'{work_dir}/splits'

import os
if not os.path.exists(splits_dir):
    os.mkdir(splits_dir)

#### 读取源Excel到Pandas

In [2]:
import pandas as pd

In [10]:
df_source = pd.read_excel(f'{work_dir}/A2-Krusty_Burger_Report_Data.xlsx')

In [11]:
df_source.head()

,Staff ID,Family Name,Other Names,Timesheet ID,Date,Start Time,End Time,Store ID,Location,State,Role Code,Role Name,Rate,Total,Total Pay
0,100001.0,SIMPSON,Bartholomew JoJo,2022-23456,2022-08-23,15:00:00,19:00:00,NSW0001,Macquarie Park,NSW,F01,Food Preparer L1,21.38,85.52,171.04
1,NaN,NaN,NaN,NaN,2022-08-25,15:00:00,19:00:00,NSW0001,Macquarie Park,NSW,F01,Food Preparer L1,21.38,85.52,NaN
2,100002.0,ALBERTSON,Kumiko,2022-23459,2022-08-27,07:00:00,12:00:00,NSW0002,Epping,NSW,S02,Front-end Service L2,22.67,113.35,269.29
3,NaN,NaN,NaN,NaN,NaT,12:30:00,14:30:00,NSW0002,Epping,NSW,S02,Front-end Service L2,22.67,45.34,NaN
4,NaN,NaN,NaN,NaN,2022-08-28,13:00:00,18:00:00,NSW0001,Macquarie Park,NSW,F02,Food Preparer L2,22.12,110.60,NaN


In [13]:
df_source.shape

(15, 15)

In [16]:
df_source.index

RangeIndex(start=0, stop=15, step=1)

In [15]:
total_row_count=df_source.shape[0]
total_row_count

15

#### 一、将一个大Excel等份拆成多个Excel
1. 使用df.iloc方法，将一个大的dataframe，拆分成多个小dataframe
2. 将使用dataframe.to_excel保存每个小excel

#### 首先填充缺失行 （复习07）

In [21]:
df_source.loc[:,['Staff ID','Family Name','Other Names','Timesheet ID']]=df_source[['Staff ID','Family Name','Other Names','Timesheet ID']].fillna(method='ffill')
df_source.head()

,Staff ID,Family Name,Other Names,Timesheet ID,Date,Start Time,End Time,Store ID,Location,State,Role Code,Role Name,Rate,Total,Total Pay
0,100001.0,SIMPSON,Bartholomew JoJo,2022-23456,2022-08-23,15:00:00,19:00:00,NSW0001,Macquarie Park,NSW,F01,Food Preparer L1,21.38,85.52,171.04
1,100001.0,SIMPSON,Bartholomew JoJo,2022-23456,2022-08-25,15:00:00,19:00:00,NSW0001,Macquarie Park,NSW,F01,Food Preparer L1,21.38,85.52,NaN
2,100002.0,ALBERTSON,Kumiko,2022-23459,2022-08-27,07:00:00,12:00:00,NSW0002,Epping,NSW,S02,Front-end Service L2,22.67,113.35,269.29
3,100002.0,ALBERTSON,Kumiko,2022-23459,NaT,12:30:00,14:30:00,NSW0002,Epping,NSW,S02,Front-end Service L2,22.67,45.34,NaN
4,100002.0,ALBERTSON,Kumiko,2022-23459,2022-08-28,13:00:00,18:00:00,NSW0001,Macquarie Park,NSW,F02,Food Preparer L2,22.12,110.60,NaN


#### 1、计算拆分后的每个excel的行数

In [52]:
# 这个大的excel，会拆分给几个人
user_names=['ted','nathan','benjamin','anny','mike']

In [53]:
# 每个人的任务数目
split_size=total_row_count//len(user_names)
if total_row_count%len(user_names)!=0:
    split_size+=1
    
split_size

3

#### 2、拆成多个dataframe

In [54]:
df_subs=[]
for idx,user_name in enumerate(user_names):
    # iloc的开始索引
    begin=idx*split_size
    # iloc的结束索引
    end=begin+split_size
    # 实现df按照iloc拆分
    df_sub=df_source.iloc[begin:end]
    # 将每个df_sub存入列表
    df_subs.append((idx,user_name,df_sub))

#### 3、将每个dataframe存入excel

In [55]:
for idx,user_name,df_sub in df_subs:
    file_name=f'{splits_dir}/A2-Krusty_Burger_Report_{idx}_{user_name}.xlsx'
    # .to_excel生成excel文件
    df_sub.to_excel(file_name,index=False)

#### 二、合并多个小Excel到一个大Excel
1. 遍历文件夹，得到要合并的Excel文件列表
2. 分别读取到dataframe，给每个df添加一列用于标记来源
3. 使用pd.concat进行df批量合并
4. 将合并后的dataframe输出到excel

##### 1. 遍历文件夹，得到要合并的Excel名称列表

In [56]:
import os
excel_names = []
for excel_name in os.listdir(splits_dir):
    excel_names.append(excel_name)
    
excel_names

['A2-Krusty_Burger_Report_0_ted.xlsx',
 'A2-Krusty_Burger_Report_1_nathan.xlsx',
 'A2-Krusty_Burger_Report_2_benjamin.xlsx',
 'A2-Krusty_Burger_Report_3_anny.xlsx',
 'A2-Krusty_Burger_Report_4_mike.xlsx']

##### 2. 分别读取到dataframe

In [57]:
df_list=[]

for excel_name in excel_names:
    # 读取每个excel到df：
    excel_path=f'{splits_dir}/{excel_name}'
    df_split=pd.read_excel(excel_path)
    # 得到username
    username=excel_name.replace('A2-Krusty_Burger_Report_','').replace('.xlsx','')[2:]
    print(excel_name,username)
    # 给每个df添加一列，即用户名字
    df_split['username']=username
    
    df_list.append(df_split)

A2-Krusty_Burger_Report_0_ted.xlsx ted
A2-Krusty_Burger_Report_1_nathan.xlsx nathan
A2-Krusty_Burger_Report_2_benjamin.xlsx benjamin
A2-Krusty_Burger_Report_3_anny.xlsx anny
A2-Krusty_Burger_Report_4_mike.xlsx mike


##### 3. 使用pd.concat进行合并

In [58]:
df_merged=pd.concat(df_list)

In [59]:
df_merged.shape

(15, 16)

In [60]:
df_merged.head()

,Staff ID,Family Name,Other Names,Timesheet ID,Date,Start Time,End Time,Store ID,Location,State,Role Code,Role Name,Rate,Total,Total Pay,username
0,100001,SIMPSON,Bartholomew JoJo,2022-23456,2022-08-23,15:00:00,19:00:00,NSW0001,Macquarie Park,NSW,F01,Food Preparer L1,21.38,85.52,171.04,ted
1,100001,SIMPSON,Bartholomew JoJo,2022-23456,2022-08-25,15:00:00,19:00:00,NSW0001,Macquarie Park,NSW,F01,Food Preparer L1,21.38,85.52,NaN,ted
2,100002,ALBERTSON,Kumiko,2022-23459,2022-08-27,07:00:00,12:00:00,NSW0002,Epping,NSW,S02,Front-end Service L2,22.67,113.35,269.29,ted
0,100002,ALBERTSON,Kumiko,2022-23459,NaT,12:30:00,14:30:00,NSW0002,Epping,NSW,S02,Front-end Service L2,22.67,45.34,NaN,nathan
1,100002,ALBERTSON,Kumiko,2022-23459,2022-08-28,13:00:00,18:00:00,NSW0001,Macquarie Park,NSW,F02,Food Preparer L2,22.12,110.60,NaN,nathan


In [61]:
df_merged['username'].value_counts()

ted         3
nathan      3
benjamin    3
anny        3
mike        3
Name: username, dtype: int64

##### 4.将合并后的dataframe输出到excel

In [63]:
df_merged.to_excel(f'{work_dir}/A2-Krusty_Burger_Report_merged.xlsx',index=False)